# Lab4
## import

In [60]:
import pandas as pd
import numpy as np

In [61]:
data = pd.read_csv("jester-data-1.csv", header=None)

In [62]:
#user_ratings = data.values
user_ratings = data.values[0:100]

In [63]:
user_ratings_vali = np.zeros(user_ratings.shape) + 99.0
user_ratings_copy = user_ratings.copy() 
count = 0
for x in range(0,user_ratings.shape[0]):
    for y in range(0,user_ratings.shape[1]):
        if (user_ratings_copy[x,y]!=99.0):
            if (np.random.random() > 0.9):
                count += 1
                temp = user_ratings_copy[x,y]
                user_ratings_copy[x,y] = user_ratings_vali[x,y]
                user_ratings_vali[x,y] = temp
                #print(temp)
print(user_ratings_vali,'\n', count,'\n', count/(user_ratings.shape[0]*user_ratings.shape[1]))

[[99.   99.   99.   ... 99.   99.   99.  ]
 [99.   99.   99.   ... 99.   99.    1.07]
 [49.   99.   99.   ... 99.   99.   99.  ]
 ...
 [99.   99.   99.   ... 99.   99.   99.  ]
 [99.   99.   99.   ... 99.    1.17 99.  ]
 [99.   99.   99.   ... 99.   99.   99.  ]] 
 791 
 0.07831683168316832


In [64]:
n_features = 10
latent_user_preferences = np.random.random((user_ratings.shape[0], n_features))
latent_item_features = np.random.random((user_ratings.shape[1],n_features))
user_preferences_weights = np.random.random((user_ratings_copy.shape[0], latent_user_preferences.shape[1] ))
item_features_weights = np.random.random((user_ratings_copy.shape[1], latent_item_features.shape[1]))

In [65]:
print(user_preferences_weights.shape, item_features_weights.shape)
print(latent_user_preferences.shape, latent_item_features.shape)

(100, 10) (101, 10)
(100, 10) (101, 10)


latent_user_preferences   <-  user_features  
latent_item_features      <-  item_features  
user_preferences_weights  <-  user_features_weights  
item_features_weights     <-  item_features_weights  

In [66]:
def predict_rating(user_id,item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    
    user_score = user_preferences_weights[user_id].dot(latent_user_preferences[user_id])
    item_score = item_features_weights[item_id].dot(latent_item_features[item_id])
    #print user_preference.dot(item_preference), user_score, item_score
    return user_preference.dot(item_preference) + user_score + item_score

def train(user_id, item_id, rating,alpha = 0.001, 
                                   latent_feature_weight_decay = 0.1, 
                                   user_weight_decay = 0.01,
                                   item_weight_decay = 0.0001):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  ( prediction_rating - rating );
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err *  ( latent_item_features[item_id] + latent_feature_weight_decay*latent_user_preferences[user_id])
    latent_item_features[item_id] -= alpha * err * ( user_pref_values + latent_feature_weight_decay*latent_item_features[item_id])
    
    user_preferences_weights[user_id] -=alpha * err *(  latent_user_preferences[user_id] + user_weight_decay* user_preferences_weights[user_id])
    item_features_weights[item_id] -=alpha * err * ( item_features_weights[item_id] + item_weight_decay* item_features_weights[item_id])
    
    
    return err
    


def sgd(iterations = 1000):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(0,iterations):
        error = []
        for user_id in range(0,latent_user_preferences.shape[0]):
            for item_id in range(0,latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if(rating != 99.0):
                    err = train(user_id,item_id,rating)
                    error.append(err)
    mse = (np.array(error) ** 2).mean()          
    print (mse)
                 

In [67]:
for i in range(0,10): 
    sgd()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:23: RuntimeWarning: overflow encountered in multiply
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:25: RuntimeWarning: overflow encountered in multiply
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in add
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:22: RuntimeWarning: overflow encountered in multiply
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:26: RuntimeWarning: overflow encountered in multiply


nan


KeyboardInterrupt: 